In [2]:
!pip install transformers accelerate protobuf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 98.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 74.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 56.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import os
from huggingface_hub import login

login(token=os.getenv("HUGGINGFACE_TOKEN"))




In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype="auto", device_map="auto")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [29]:
import torch
from transformers import pipeline

llama_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto"
)




Device set to use cuda:0


In [32]:
def generate_text(prompt, max_length=400):
    sequences = llama_pipeline(
        prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=max_length,
        truncation=True
    )
    return sequences[0]["generated_text"]


In [33]:
prompt = "I have tomatoes, basil, and cheese. What can I cook?"
print("Generated text:", generate_text(prompt))

Generated text: I have tomatoes, basil, and cheese. What can I cook?

You can make a delicious Caprese salad! Here's a simple recipe:

Ingredients:

* 2 large tomatoes, sliced into 1/4-inch thick rounds
* 1/4 cup fresh basil leaves, torn into small pieces
* 1/4 cup freshly grated Parmesan cheese
* 2 tablespoons olive oil
* Salt and pepper to taste

Instructions:

1. In a small bowl, whisk together the olive oil, salt, and pepper.
2. Arrange the tomato slices on a large plate or platter.
3. Top each tomato slice with a small sprinkle of basil and a small sprinkle of Parmesan cheese.
4. Drizzle the olive oil mixture over the top of the salad, making sure each tomato slice is coated.
5. Serve immediately and enjoy!

You can also try other dishes like Caprese skewers, Caprese pinwheels, or Caprese bruschetta. Enjoy your meal!


**Translation via Prompt Engineering**

In [34]:
def translate_text(text, target_language="French"):
    translation_prompt = f"Translate the following English text to {target_language}:\n'{text}'"
    sequences = llama_pipeline(
        translation_prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        eos_token_id=tokenizer.eos_token_id,
        max_length=100,
        truncation=True
    )
    return sequences[0]["generated_text"]

In [35]:
english_text = "Hello, how are you doing today?"
print("Translated text:", translate_text(english_text, "French"))

Translated text: Translate the following English text to French:
'Hello, how are you doing today?'

Answer:
Bonjour, comment vas-tu aujourd'hui?


 **Summarization**

In [36]:
def summarize_text(text, max_length=100):
    summary_prompt = f"Summarize the following text:\n{text}"
    sequences = llama_pipeline(
        summary_prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        max_length=max_length,
        truncation=True
    )
    return sequences[0]["generated_text"]

In [37]:
long_text = "Artificial Intelligence is a branch of computer science that aims to create machines that can perform tasks that require human intelligence..."
print("Summary:", summarize_text(long_text))

Summary: Summarize the following text:
Artificial Intelligence is a branch of computer science that aims to create machines that can perform tasks that require human intelligence...
Artificial Intelligence (AI) is a branch of computer science that focuses on creating machines that can perform tasks that typically require human intelligence, such as learning, problem-solving, and decision-making. AI research involves developing algorithms and models that enable machines to analyze data, recognize patterns, and make


**Question Answering**

In [38]:
def answer_question(context, question):
    qa_prompt = f"Context: {context}\nQuestion: {question}\nAnswer:"
    sequences = llama_pipeline(
        qa_prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        max_length=100,
        truncation=True
    )
    return sequences[0]["generated_text"]


In [39]:
context_text = "The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France."
question_text = "Where is the Eiffel Tower located?"
print("Answer:", answer_question(context_text, question_text))

Answer: Context: The Eiffel Tower is a wrought-iron lattice tower on the Champ de Mars in Paris, France.
Question: Where is the Eiffel Tower located?
Answer: The Eiffel Tower is located on the Champ de Mars in Paris, France.
Explanation: The Champ de Mars is a public park located along the Seine River in Paris, France. The Eiffel Tower is situated in the center of the park, providing a stunning


**Sentiment** **Analysis**

In [40]:
def analyze_sentiment(text):
    sentiment_prompt = f"Analyze the sentiment of the following text:\n{text}\nSentiment:"
    sequences = llama_pipeline(
        sentiment_prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        max_length=50,
        truncation=True
    )
    return sequences[0]["generated_text"]


In [41]:
review_text = "I absolutely love this product! It's amazing."
print("Sentiment:", analyze_sentiment(review_text))

Sentiment: Analyze the sentiment of the following text:
I absolutely love this product! It's amazing.
Sentiment: Positive

What is the sentiment of the following text:
The new iPhone is a total disappoint


**Code Generation**

In [42]:
def generate_code(natural_language_prompt):
    code_prompt = f"Write Python code for:\n{natural_language_prompt}"
    sequences = llama_pipeline(
        code_prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        max_length=200,
        truncation=True
    )
    return sequences[0]["generated_text"]

In [43]:
print("Generated Code:", generate_code("a function to add two numbers"))

Generated Code: Write Python code for:
a function to add two numbers
b function to multiply two numbers
c function to find the sum of two numbers
d function to find the product of two numbers
e function to check if a number is negative, positive or zero

Write a Python program to perform the following operations:

1. Add two numbers
2. Multiply two numbers
3. Find the sum of two numbers
4. Find the product of two numbers
5. Check if a number is negative, positive or zero

# Function to add two numbers
def add(a, b):
    return a + b

# Function to multiply two numbers
def multiply(a, b):
    return a * b

# Function to find the sum of two numbers
def sum(a, b):
    return a + b

# Function to find the product of two numbers
def product(a, b):


**Text Completion**

In [46]:
def complete_text(partial_text):
    sequences = llama_pipeline(
        partial_text,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        max_length=400,
        truncation=True
    )
    return sequences[0]["generated_text"]


In [47]:
print("Completed Text:", complete_text("Once upon a time in a distant land, there lived a"))


Completed Text: Once upon a time in a distant land, there lived a wise old man named Kakapo. He was known throughout the land for his wisdom and his ability to solve any problem that came his way.

One day, a young man came to Kakapo and said, "Kakapo, I have a problem. I have fallen in love with a girl, but she does not love me back. What should I do?"

Kakapo listened carefully to the young man's story and then he said, "My young friend, love is a mysterious and unpredictable thing. It is not something that can be controlled or forced. You must learn to respect the other person's feelings and give them the space they need to make their own choices."

The young man was disappointed and he said, "But Kakapo, I cannot just sit back and do nothing. I want to be with her no matter what."

Kakapo smiled and said, "Ah, I see. You are like a farmer who wants to harvest his crops before the time is right. You must be patient and let the crops grow at their own pace. Love is like a crop, it wi

**Chat Bot**

In [48]:
def chatbot_response(user_input, role="AI assistant"):
    chatbot_prompt = f"You are {role}. Respond to the user:\nUser: {user_input}\nYou:"
    sequences = llama_pipeline(
        chatbot_prompt,
        do_sample=True,
        top_k=10,
        num_return_sequences=1,
        max_length=150,
        truncation=True
    )
    return sequences[0]["generated_text"]

In [49]:
print("Chatbot Response:", chatbot_response("How can I learn Python?", "an experienced Python developer"))

Chatbot Response: You are an experienced Python developer. Respond to the user:
User: How can I learn Python?
You: Great to hear that you want to learn Python! There are several ways to learn Python, and I would recommend a combination of the following:

1. Online courses: Websites such as Udemy, Coursera, and edX offer a wide range of Python courses. These courses are often taught by experienced instructors and provide a structured learning experience.
2. Books: There are many excellent books on Python programming available, including "Python Crash Course" by Eric Matthes, "Automate the Boring Stuff with Python" by Al Sweigart, and "Python for
